In [1]:
from rodan_pre.rodan_pretrained import RodanPretrained
model = RodanPretrained(pretrained_lr=1e-4, my_layers_lr=1e-3, warmup_steps = 10000)

In [2]:
from dataloading2 import nanopore_datamodule
from data_utils.split_methods import get_kfold_split_func, get_default_split
dm = nanopore_datamodule(
    pos_files='pos_2022', 
    neg_files='neg_2022', 
    verbose=1, 
    batch_size=64, 
    valid_limit=10000, 
    workers=32,
    split_method=get_kfold_split_func(total_k=5, current_k=0),
    window=4096,
)

valid files indicies
[3, 5, 6, 9, 32, 34, 42, 45, 48, 50, 61, 65, 66, 73, 78, 79, 82, 84, 95, 104, 106, 110, 113, 115, 128, 129, 131, 138, 139, 144, 151, 156, 159, 161, 177, 187, 188, 189, 199, 202, 205, 211, 219, 227, 232, 234, 235, 238, 239, 240, 252, 254, 257, 267, 268]
[2, 7, 8, 9, 10, 13, 17, 24, 32, 33, 36, 42, 44, 46, 48, 58, 64, 71, 74, 78, 86, 87, 88, 92, 93, 101, 103, 105, 117, 123, 137, 149, 159, 164, 165, 166, 173, 177, 180, 182, 185, 190, 191, 193, 198, 221, 228, 231, 233, 236, 241, 247, 251, 252, 256, 263, 264, 269, 276, 285, 295, 301, 302, 303, 325, 331, 348, 349, 350, 354, 358, 359, 363, 365, 372, 373, 375, 377, 378, 381, 387, 388, 406, 407, 409, 410, 412, 415, 421, 424, 448, 454, 457, 467, 472, 475, 480, 481, 484, 488, 493, 498, 507, 513]
train files indicies
[0, 1, 2, 4, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 35, 36, 37, 38, 39, 40, 41, 43, 44, 46, 47, 49, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 62, 63, 64, 67

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import CometLogger
from pytorch_lightning.callbacks import ModelCheckpoint

experiment_name = 'rodan_2022_kfold_0of5'
checkpoint_callback = ModelCheckpoint(dirpath=f"checkpoints_pl/{experiment_name}", save_top_k=2, monitor="valid_loss")
logger = CometLogger(api_key="TEVQbgxxvilM1WdTyqZLJ57ac", project_name='RNAModif', experiment_name=experiment_name) 
trainer= pl.Trainer(
    max_steps = 1000000, logger=logger, accelerator='gpu',
    auto_lr_find=False, val_check_interval=500,  
    log_every_n_steps=500, benchmark=True, precision=16,
    callbacks=[checkpoint_callback])#, profiler="simple")

trainer.fit(model, dm)


CometLogger will be initialized in online mode
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type            | Params
------------------------------------------
0 | model | Sequential      | 10.9 M
1 | acc   | Accuracy        | 0     
2 | cm    | ConfusionMatrix | 0     
------------------------------------------
10.9 M    Trainable params
0         Non-trainable params
10.9 M    Total params
21.760    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/jovyan/my-conda-envs/RNAmodEnvRod/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:219: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/jovyan/my-conda-envs/RNAmodEnvRod/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: 2 nan values found in confusion matrix have been replaced with zeros.
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

/home/jovyan/my-conda-envs/RNAmodEnvRod/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experime

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]